In [1]:
import pandas as pd
from keras.callbacks import History, ReduceLROnPlateau,EarlyStopping,ModelCheckpoint
import os
import numpy as np
from data_analysis import calculate_metrics, load_weights_and_evaluate
from model_builders import GCN_pretraining
from hyperparameter_tuning_GCN import objective
from functools import partial
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials
import pickle

Using TensorFlow backend.
C:\Users\user\Miniconda3\lib\site-packages\sklearn\externals\joblib\__init__.py:15: DeprecationWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=DeprecationWarning)


In [2]:
# Specify model callbacks on training
es = EarlyStopping(monitor='loss',patience=8, min_delta=0)
rlr = ReduceLROnPlateau(monitor='loss',factor=0.5, patience=4, verbose=1, min_lr=0.0000001)

model_params = {
        "num_layers" : 3,
        "max_atoms" : 70,
        "num_atom_features" : 62,
        "num_atom_features_original" : 62,
        "num_bond_features" : 6,
        "max_degree" : 5,
        "conv_width" : [32,64,96],
        "fp_length" : [96,96,96],
        "activ_enc" : "selu",
        "activ_dec" : "selu",
        "learning_rates" : [0.001,0.001,0.001],
        "learning_rates_fp": [0.005,0.005,0.005],
        "losses_conv" : {
                    "neighbor_output": "mean_squared_error",
                    "self_output": "mean_squared_error",
                    },
        "lossWeights" : {"neighbor_output": 1.0, "self_output": 1.0},
        "metrics" : "mse",
        "loss_fp" : "mean_squared_error",
        "enc_layer_names" : ["enc_1", "enc_2", "enc_3"],
        'callbacks' : [es,rlr],
        'adam_decay': 0.0005329142291371636,
        'beta': 5,
        'p': 0.004465204118126482,
        'dense_size' : [96,96,48],
        'dropout_rate' : [0.1,0.1],
        'lr' : 0.001,
        'batch_size' : int(64),
        'n_epochs' : int(5)
        }


In [3]:
fspace = {
    'conv1' : hp.quniform('conv1', 32, 96, 8),
    'conv2' : hp.quniform('conv2', 48, 128, 8),
    'conv3' : hp.quniform('conv3', 64, 168, 8),
    'fp' : hp.quniform('fp', 64, 196, 8),
    'dense1' : hp.quniform('dense1',96,256,32),
    'dense2' : hp.quniform('dense2',96,256,32),
    'dense3' : hp.quniform('dense3',48,128,32),
    'dropout_rate' : hp.uniform('dropout_rate',0.1,0.5),
    'lr' : hp.uniform('lr',0.0005,0.01),
    'n_epochs' : hp.quniform('n_epochs',15,40,5) 
}

In [4]:
base_path = 'C:/Users/user/Documents/kinase_binding/learning/'
target = 'p38'
model_name = 'gcn_pretraining_ensemble'

In [5]:
# no need for manual changes in this cell
train_files = os.path.join(base_path, f'data/{target}/split_aveb/fold_{{}}/train_{{}}.csv')
val_files = os.path.join(base_path, f'data/{target}/split_aveb/fold_{{}}/val_{{}}.csv')
weight_files = os.path.join(base_path, f'results/{target}/{model_name}/fold_{{}}/model_weights/model_{{}}.h5')

In [6]:
train_sets = [pd.read_csv(train_files.format(i,i), index_col = 0) if 'Unnamed: 0' in pd.read_csv(train_files.format(i,i)) else pd.read_csv(train_files.format(i,i)) for i in range(7)]
val_sets = [pd.read_csv(val_files.format(i,i), index_col = 0) if 'Unnamed: 0' in pd.read_csv(val_files.format(i,i)) else pd.read_csv(val_files.format(i,i)) for i in range(7)]
test_set = pd.read_csv(os.path.join(base_path, f'data/{target}/split_aveb/test.csv')) 

In [7]:
fmin_objective = partial(objective, train_sets = train_sets, val_sets = val_sets)

In [8]:
def run_trials():

    trials_step = 10  # how many additional trials to do after loading saved trials. 1 = save after iteration
    max_trials = 2  # initial max_trials. put something small to not have to wait

    
    try:  # try to load an already saved trials object, and increase the max
        trials = pickle.load(open("my_model.hyperopt", "rb"))
        print("Found saved Trials! Loading...")
        max_trials = len(trials.trials) + trials_step
        print("Rerunning from {} trials to {} (+{}) trials".format(len(trials.trials), max_trials, trials_step))
    except:  # create a new trials object and start searching
        trials = Trials()

    best = fmin(fn = fmin_objective, space = fspace, algo=tpe.suggest, max_evals=max_trials, trials=trials)

    print("Best:", best)
    
    # save the trials object
    with open("my_model.hyperopt", "wb") as f:
        pickle.dump(trials, f)
    return(trials)

In [9]:
trials = run_trials()

Found saved Trials! Loading...
Rerunning from 90 trials to 93 (+3) trials
LAYER 0                                                                                                                
LAYER 1                                                                                                                
LAYER 2                                                                                                                
 97%|███████████████████████████████████████████████████████████████▊  | 90/93 [00:05<00:00, 16.30trial/s, best loss=?]

C:\Users\user\Miniconda3\lib\site-packages\tensorflow\python\ops\gradients_impl.py:112: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "

C:\Users\user\Miniconda3\lib\site-packages\tensorflow\python\ops\gradients_impl.py:112: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "



LAYER 0                                                                                                                
LAYER 1                                                                                                                
LAYER 2                                                                                                                
 97%|███████████████████████████████████████████████████████████████▊  | 90/93 [00:53<00:01,  1.68trial/s, best loss=?]

C:\Users\user\Miniconda3\lib\site-packages\tensorflow\python\ops\gradients_impl.py:112: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "

C:\Users\user\Miniconda3\lib\site-packages\tensorflow\python\ops\gradients_impl.py:112: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "



LAYER 0                                                                                                                
LAYER 1                                                                                                                
LAYER 2                                                                                                                
 97%|███████████████████████████████████████████████████████████████▊  | 90/93 [01:43<00:03,  1.15s/trial, best loss=?]

...

LAYER 0                                                                                                                
LAYER 1                                                                                                                
LAYER 2                                                                                                                
 99%|█████████████████████████████████████████████▌| 92/93 [26:09<07:26, 446.82s/trial, best loss: -0.6526830105025258]

C:\Users\user\Miniconda3\lib\site-packages\tensorflow\python\ops\gradients_impl.py:112: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "

C:\Users\user\Miniconda3\lib\site-packages\tensorflow\python\ops\gradients_impl.py:112: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "



LAYER 0                                                                                                                
LAYER 1                                                                                                                
LAYER 2                                                                                                                
 99%|█████████████████████████████████████████████▌| 92/93 [28:26<07:26, 446.82s/trial, best loss: -0.6526830105025258]

C:\Users\user\Miniconda3\lib\site-packages\tensorflow\python\ops\gradients_impl.py:112: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "

C:\Users\user\Miniconda3\lib\site-packages\tensorflow\python\ops\gradients_impl.py:112: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "



100%|██████████████████████████████████████████████| 93/93 [30:38<00:00, 569.05s/trial, best loss: -0.6526830105025258]
Best: {'conv1': 88.0, 'conv2': 96.0, 'conv3': 144.0, 'dense1': 224.0, 'dense2': 192.0, 'dense3': 64.0, 'dropout_rate': 0.35233415527455836, 'fp': 88.0, 'lr': 0.0005039863454731193, 'n_epochs': 40.0}


In [12]:
trials.trials[92]

{'state': 2,
 'tid': 92,
 'spec': None,
 'result': {'loss': -0.589932674881396, 'status': 'ok'},
 'misc': {'tid': 92,
  'cmd': ('domain_attachment', 'FMinIter_Domain'),
  'workdir': None,
  'idxs': {'conv1': [92],
   'conv2': [92],
   'conv3': [92],
   'dense1': [92],
   'dense2': [92],
   'dense3': [92],
   'dropout_rate': [92],
   'fp': [92],
   'lr': [92],
   'n_epochs': [92]},
  'vals': {'conv1': [72.0],
   'conv2': [48.0],
   'conv3': [120.0],
   'dense1': [160.0],
   'dense2': [128.0],
   'dense3': [128.0],
   'dropout_rate': [0.3509845876629733],
   'fp': [88.0],
   'lr': [0.0056318721061324635],
   'n_epochs': [25.0]}},
 'exp_key': None,
 'owner': None,
 'version': 0,
 'book_time': datetime.datetime(2020, 5, 8, 9, 45, 19, 359000),
 'refresh_time': datetime.datetime(2020, 5, 8, 9, 59, 33, 546000)}

In [10]:
run_trials()

Found saved Trials! Loading...
Rerunning from 65 trials to 68 (+3) trials
LAYER 0                                                                                                                
LAYER 1                                                                                                                
LAYER 2                                                                                                                
 96%|███████████████████████████████████████████████████████████████   | 65/68 [00:08<00:00,  8.02trial/s, best loss=?]

C:\Users\user\Miniconda3\lib\site-packages\tensorflow\python\ops\gradients_impl.py:112: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "

C:\Users\user\Miniconda3\lib\site-packages\tensorflow\python\ops\gradients_impl.py:112: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "



LAYER 0                                                                                                                
LAYER 1                                                                                                                
LAYER 2                                                                                                                
 96%|███████████████████████████████████████████████████████████████   | 65/68 [02:45<00:07,  2.54s/trial, best loss=?]

C:\Users\user\Miniconda3\lib\site-packages\tensorflow\python\ops\gradients_impl.py:112: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "

C:\Users\user\Miniconda3\lib\site-packages\tensorflow\python\ops\gradients_impl.py:112: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "



LAYER 0                                                                                                                
LAYER 1                                                                                                                
LAYER 2                                                                                                                
 96%|███████████████████████████████████████████████████████████████   | 65/68 [05:29<00:15,  5.06s/trial, best loss=?]

...

LAYER 0                                                                                                                
LAYER 1                                                                                                                
LAYER 2                                                                                                                
 99%|██████████████████████████████████████████▎| 67/68 [1:11:56<22:45, 1365.04s/trial, best loss: -0.6526830105025258]

C:\Users\user\Miniconda3\lib\site-packages\tensorflow\python\ops\gradients_impl.py:112: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "

C:\Users\user\Miniconda3\lib\site-packages\tensorflow\python\ops\gradients_impl.py:112: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "



LAYER 0                                                                                                                
LAYER 1                                                                                                                
LAYER 2                                                                                                                
 99%|██████████████████████████████████████████▎| 67/68 [1:16:49<22:45, 1365.04s/trial, best loss: -0.6526830105025258]

C:\Users\user\Miniconda3\lib\site-packages\tensorflow\python\ops\gradients_impl.py:112: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "

C:\Users\user\Miniconda3\lib\site-packages\tensorflow\python\ops\gradients_impl.py:112: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "



100%|███████████████████████████████████████████| 68/68 [1:21:49<00:00, 1562.10s/trial, best loss: -0.6526830105025258]
Best: {'conv1': 88.0, 'conv2': 96.0, 'conv3': 144.0, 'dense1': 224.0, 'dense2': 192.0, 'dense3': 64.0, 'dropout_rate': 0.35233415527455836, 'fp': 88.0, 'lr': 0.0005039863454731193, 'n_epochs': 40.0}
